In [ ]:
# This notebook shows how to get shape information from the
# echoSMs anatomical datastore, format it appropriately for
# an echoSMs KRM model, then calculate the TS.

import echosms
import requests  # for the web API calls
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# The location of the echoSMs datastore server
baseURI = 'https://echosms-data-store-app-ogogm.ondigitalocean.app/'

# You'll need to have started the web API on your local hackathon server for this one to work
# baseURI = 'https://127.0.0.1:8000'

# Get all the datastore organisms in the CLAY_HORNE dataset.
# This returns the metadata about the specimens but no shape information - the shapes will
# be requested individually later
r = requests.get(baseURI + 'v2/specimens/?dataset_id=CLAY_HORNE')
if r.status_code != 200:
    print('Failed to get specimen metadata from the datastore')

In [ ]:
# Create an instance of the echoSMs KRM scattering model
m = echosms.KRMModel()

# Loop over each specimen in the CLAY_HORNE dataset (there are 4)
for o in r.json():
    print(f'Processing specimen {o["specimen_id"]} from the {o["dataset_id"]} dataset')

    # Get the organism data (including the shape) from the datastore
    r = requests.get(baseURI + 'v2/specimen/' + o['id'] + '/data')
    if r.status_code != 200:
        print(f'Request for data from specimen {o["id"]} failed - skipping')
        continue

    # Get the model data out of the requests object. The data
    # ends up in a Python dictionary
    s = r.json()

    # Assemble the echoSMs model parameters. We use an echoSMs 
    # utility function to convert the datastore shape data structure
    # into what the KRM model requires.
    p = {'medium_c': 1500,  # [m/s]
         'medium_rho': 1024,  # [kg/m^3]
         'theta': 90.0,  # [deg] dorsal aspect
         'f': np.arange(10, 201, 1)*1e3,  # [Hz]
         'organism': echosms.krmorganism_from_datastore(s['shapes'])}

    # Calculate the TS using those parameters
    ts = m.calculate_ts(p)

    # Add to a plot of all the TS results
    plt.plot(p['f']*1e-3, ts, label=s['specimen_id'])

# Annotate the plot
plt.legend(title='Specimens')
plt.title('Dataset ' + s['dataset_id'])
plt.xlabel('Frequency [kHz]')
plt.ylabel('TS [dB re 1 m$^2$]')
plt.show()
